In [1]:
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys, os
sys.path.insert(0, "/home/storage/hans/jax_reco")

import logging
logging.getLogger("jax").setLevel(logging.ERROR)

In [2]:
from lib.sim_data_i3 import I3SimHandlerFtr
from lib.geo import center_track_pos_and_time_based_on_data

from lib.network import get_network_eval_v_fn
#from lib.network import get_network_matmul_eval_v_fn as get_network_eval_v_fn

from dom_track_eval import get_eval_network_doms_and_track
from time_sampler import sample_times_clean
from likelihood_const_vertex import get_neg_mpe_llh_const_vertex

2024-05-28 13:09:53.578741: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
key = jax.random.PRNGKey(2)

dtype = jnp.float32
#dtype = jnp.float64

In [4]:
eval_network_v = get_network_eval_v_fn(bpath='/home/storage/hans/jax_reco/data/network', dtype=dtype)
eval_network_doms_and_track = get_eval_network_doms_and_track(eval_network_v, dtype=dtype)

In [5]:
bp = '/home/storage2/hans/i3files/21217'
sim_handler = I3SimHandlerFtr(os.path.join(bp, 'meta_ds_21217_from_35000_to_53530.ftr'),
                              os.path.join(bp, 'pulses_ds_21217_from_35000_to_53530.ftr'),
                              '/home/storage/hans/jax_reco/data/icecube/detector_geometry.csv')

# Get a simulated muon event.
event_index = 5
meta, pulses = sim_handler.get_event_data(event_index)
print(f"muon energy: {meta['muon_energy_at_detector']/1.e3:.1f} TeV")

# Get dom locations, first hit times, and total charges (for each dom).
event_data = sim_handler.get_per_dom_summary_from_sim_data(meta, pulses)

print("n_doms", len(event_data))

muon energy: 3.4 TeV
n_doms 43


In [6]:
# Let's generate some new first hit times following our triple pandel model.
# (avoid problems with time smearing for now -> to be implemented: gaussian convoluted triple pandel.)

track_pos = jnp.array([meta['muon_pos_x'], meta['muon_pos_y'], meta['muon_pos_z']])
track_time = meta['muon_time']
track_zenith = meta['muon_zenith']
track_azimuth = meta['muon_azimuth']
track_src = jnp.array([track_zenith, track_azimuth])

print("old track vertex:", track_pos)
centered_track_pos, centered_track_time = center_track_pos_and_time_based_on_data(event_data, track_pos, track_time, track_src)
print("new track vertex:", centered_track_pos)

key, subkey = jax.random.split(key)
first_times = sample_times_clean(eval_network_doms_and_track, event_data, track_pos, track_src, track_time, subkey)

old track vertex: [ 143.40123598  106.94081331 -815.20509865]
new track vertex: [ -25.42355091  403.14288541 -154.63778486]


In [7]:
# Create some n_photons from qtot (by rounding up).
n_photons = np.round(event_data['charge'].to_numpy()+0.5)

# Combine into single data tensor for fitting.
fake_event_data = jnp.column_stack([jnp.array(event_data[['x', 'y', 'z']].to_numpy()), 
                                    jnp.array(first_times), 
                                    jnp.array(n_photons)])
print(fake_event_data.shape)

# Send to GPU.
fake_event_data.devices()
centered_track_pos.devices()
centered_track_time.devices()
track_src.devices()

(43, 5)


{cuda(id=0)}

In [8]:
neg_llh = get_neg_mpe_llh_const_vertex(eval_network_doms_and_track, fake_event_data, centered_track_pos, centered_track_time)
neg_llh_v = jax.jit(jax.vmap(neg_llh, 0, 0))

In [9]:
n_eval = 200

zenith = np.linspace(track_src[0]-0.05, track_src[0]+0.05, n_eval)
azimuth = np.linspace(track_src[1]-0.05, track_src[1]+0.05, n_eval)
X, Y = np.meshgrid(zenith, azimuth)

init_dirs = np.column_stack([X.flatten(), Y.flatten()])

In [10]:
#with jax.default_matmul_precision('tensorfloat32'):
%timeit -n20 logls = neg_llh_v(init_dirs).block_until_ready()

158 ms ± 43.6 ms per loop (mean ± std. dev. of 7 runs, 20 loops each)


In [11]:
# 163 ms ± 44.2 ms per loop (mean ± std. dev. of 7 runs, 20 loops each) for jnp.float32

In [12]:
# 1.53 s ± 22.6 ms per loop (mean ± std. dev. of 7 runs, 20 loops each) for jnp.float64

In [13]:
# 168 ms ± 16.4 ms per loop (mean ± std. dev. of 7 runs, 20 loops each) // no TF32?